<a href="https://colab.research.google.com/github/amandatz/linear-programming/blob/main/Atividade3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 3



Amanda Topanotti Zanette (22100776)

**Importações e funções auxiliares**

In [6]:
using Pkg
Pkg.add("JuMP")
Pkg.add("HiGHS")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed CodecBzip2 ───────── v0.8.5
   Installed StructTypes ──────── v1.11.0
   Installed JSON3 ────────────── v1.14.3
   Installed BenchmarkTools ───── v1.6.3
   Installed MutableArithmetics ─ v1.6.7
   Installed MathOptInterface ─── v1.46.0
   Installed JuMP ─────────────── v1.29.3
    Updating `~/.julia/environments/v1.11/Project.toml`
  [4076af6c] + JuMP v1.29.3
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.3
  [523fee87] + CodecBzip2 v0.8.5
  [0f8b85d8] + JSON3 v1.14.3
  [4076af6c] + JuMP v1.29.3
  [b8f27783] + MathOptInterface v1.46.0
  [d8a4904e] + MutableArithmetics v1.6.7
  [856f2bd8] + StructTypes v1.11.0
  [9abbd945] + Profile v1.11.0
Precompiling project...
   6540.6 ms  ✓ CodecBzip2
   6791.5 ms  ✓ StructTypes
   4391.6 ms  ✓ BenchmarkTools
  18649.4 ms  ✓ MutableArithmetics
  20293.6 ms  ✓ JSON3
  82986.7 ms  ✓ MathOptInterface
 

In [7]:
using JuMP, HiGHS, LinearAlgebra, Printf

## Problema

A Pastesian é uma fábrica de massas familiar que está planejando a produção de lasanhas para os próximos 4 meses. Além do sabor tradicional, ela decidiu lançar um novo sabor de lasanha para esta temporada. A empresa planeja suas operações ao longo de 4 meses. Para o sabor tradicional, a demanda é considerada conhecida com base em dados históricos: 200, 350, 150 e 250 unidades nos meses 1, 2, 3 e 4, respectivamente.

## Parâmetros

In [8]:
# Períodos (mês)
T = 1:4;
# Custo (R$/unidade) de produção em cada mês
cp = [5.5, 7.2, 8.8, 10.9];
# Custo (R$/unidade) de manter as lasanhas em estoque, de um mês para o seguinte
ce = [1.3, 1.95, 2.2];

# Modelo para lasanha original

**Parâmetros**

In [34]:
# Demanda certa nos meses 1–3 (lasanha original)
D_o = [200, 350, 150];
# Lasanhas originais (unidade) em estoque inicialmente
e0_o = 50;

# Cenários para o mês 4
C_o = 1:3;
# Demanda incerta no mês 4
D4_o = [220, 250, 300]
# Penalidade por demanda não atendida no mês 4
pp_o = 5.045

5.045

**Modelo**

Este código constrói o modelo base contendo apenas os elementos referentes à lasanha original. Ele representa a parte fixa e comum aos dois problemas, tanto ao modelo determinístico quanto ao estocástico da lasanha nova, e, por isso, será reutilizado posteriormente na formulação de ambos.

Observe que a função objetivo ainda não é definida, pois ela difere entre os dois modelos finais. Aqui, o foco está apenas nas restrições estruturais da lasanha original. Em particular, consideramos:

- O estoque inicial do produto original: `e_o[1] = e0_o`;
- O balanço de estoque ao longo dos três primeiros meses: `e_o[t+1] = e_o[t] + l_o[t] - D_orig[t]` para os meses `t = 1,2,3`;


In [51]:
function build_base_original(model, T, D_o, e0_o, C, D4_o)
    ## ============================================================
    ## Variáveis
    ## ============================================================
    @variable(model, l_o[T] >= 0)       # produção da lasanha original
    @variable(model, e_o[T] >= 0)       # estoque da lasanha original
    @variable(model, l4_o[C] >= 0)      # produção no mês 4 por cenário
    @variable(model, dp_o[C] >= 0)      # demanda não atendida no mês 4 por cenário

    ## ============================================================
    ## Restrições (meses 1,2,3)
    ## ============================================================

    # Estoque inicial
    @constraint(model, e_o[1] == e0_o)

    # Balanço meses 1, 2, 3
    @constraint(model, BAL_o[t in T[1:end-1]],
        e_o[t+1] == e_o[t] + l_o[t] - D_o[t]
    )

    ## ============================================================
    ## Restrições (mês 4)
    ## ============================================================

    # Atender as demandas incertas no mês 4
    @constraint(model, DEM_o[c in C],
        dp_o[c] >= D4_o[c] - (l4_o[c] + e_o[4])
    )

    # No-anticipativity: a produção no mês 4 deve ser a mesma em todos os cenários
    @constraint(model, NA_o[c in C],
        sum(l4_o) == length(C) * l4_o[c]
    )

    return model, l_o, e_o, l4_o, dp_o
end

build_base_original (generic function with 3 methods)

# Modelo determinístico para lasanha nova

**Parâmetros**

In [ ]:
# Demanda certa nos meses 1–3 (lasanha nova)
D_n = [30, 70, 140];

# Cenários para o mês 4
C_o = 1:3;
# Demanda incerta no mês 4
D4_n = [200, 240, 300];
# Probabilidades
P4_n = [0.3, 0.5, 0.2];

# Capacidade total de estoque em cada mês (para as duas lasanhas somadas)
E = [200, 220, 230, 250]

In [ ]:
# Demanda média esperada da lasanha nova no mês 4
d4_mean_n = dot(D4_n, P4_n);
D_n_det = vcat(D_n_1_3, d4_mean_n);

**Modelo**

- Adicionar capacidade de estoque
([estoque lasanha antiga no mes 1] + [estoque lasanha nova no mes 1] <= 200 ...)

In [ ]:
# Capacidade de estoque
@constraint(model, CE[t = T], e[t] <= 200);

In [64]:
model_det = Model(HiGHS.Optimizer);
model_det, l_o, e_o = build_base_original(model_det, T, D_o, e0_o);

LoadError: At In[42]:7: `@variable(model, l4_o[C] >= 0)`: unexpected error parsing reference set: C

In [ ]:
@variable(model_det, l_n[T] >= 0)   # produção da lasanha nova
@variable(model_det, e_n[T] >= 0)   # estoque da lasanha nova

In [ ]:
# Estoque inicial da lasanha nova é zero
@constraint(model_det, e_n[1] == 0)

# Balanço de estoque da lasanha nova (meses 1,2,3)
@constraint(model_det, BAL_n[t in T[1:end-1]],
    e_n[t+1] == e_n[t] + l_n[t] - D_n_det[t]
)

# Atendimento à demanda da lasanha nova (de fato só precisa em t=4,
# mas deixamos para todos os meses por simetria didática)
@constraint(model_det, AD_n[t in T],
    l_n[t] + e_n[t] >= D_n_det[t]
)

In [ ]:
# ============================
# Capacidade de estoque total (original + nova)
# ============================
@constraint(model_det, CE[t in T],
    e_o[t] + e_n[t] <= E[t]
)

# Modelo estocástico para lasanha nova

In [61]:
model_estocastico = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none